# Исследование надежности заемщиков.

### Шаг 1. Обзор данных

In [1]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')
display(df)
df.info()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Шаг 2.1 Заполнение пропусков

In [2]:
print(df.isna().sum())
print()
#Столбец days_employed нам не нужен. Удаляем
del df['days_employed']
total_income_median = df['total_income'].median()
print('Доля пропущенных значений в стобце total_income: {:.2%}'.format(df['total_income'].isna().sum()
                                                                      / df['total_income'].count()))
print()
#Все значения столбца total_income заменили на медианное
df['total_income'] = df['total_income'].fillna(total_income_median) 
print(df.isna().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Доля пропущенных значений в стобце total_income: 11.23%

children            0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


<div class="alert alert-info">
<b>Комментарий:</b><br>
• Пропущеннымые значения были найдены в столбце ежемесячных доходов.<br>
• Доля пропущенных значений = 11.23%<br>
• Одна из возможных причин пропусков - никогда не работали. Потому нет дохода.<br>
• Заполняем значение в столбцах медианным значением из-за большой разнице в значениях столбца. Среднее значение нам не подойдет.
</div>

### Шаг 2.2 Проверка данных на аномалии и исправления.

In [3]:
# Проверяю уникальные значения с целью поиска аномалий и дубликатов
print(df['children'].unique()) 
print(df['dob_years'].unique())
print(df['education'].unique())
print(df['education_id'].unique())
print(df['family_status'].unique())
print(df['family_status_id'].unique())
print(df['gender'].unique())
print(df['income_type'].unique())
print(df['purpose'].unique())

[ 1  0  3  2 -1  4 20  5]
[42 36 33 32 53 27 43 50 35 41 40 65 54 56 26 48 24 21 57 67 28 63 62 47
 34 68 25 31 30 20 49 37 45 61 64 44 52 46 23 38 39 51  0 59 29 60 55 58
 71 22 73 66 69 19 72 70 74 75]
['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']
[0 1 2 3 4]
['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']
[0 1 2 3 4]
['F' 'M' 'XNA']
['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']
['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство нед

In [4]:
children_median = df['children'].median()
#Отрицательное количество детей заменил на медианное. 20 детей на 2 (возможно 0 написан случайно)
print(df[df['gender'] == 'XNA'].count())
# Удаляю строку с полом XNA. Он встречается лишь раз. На результат не повлияет сильно.
df = df[df.gender != 'XNA']
df['children'] = df['children'].replace({-1: children_median, 20: 2}).astype(int)


display(df.head(10))

children            1
dob_years           1
education           1
education_id        1
family_status       1
family_status_id    1
gender              1
income_type         1
debt                1
total_income        1
purpose             1
dtype: int64


,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Шаг 2.3. Изменение типов данных.

In [5]:
df['total_income'] = df['total_income'].astype(int)

### Шаг 2.4. Удаление дубликатов.

In [6]:
#Проверка на явные дубликаты
print('Дубликатов явных:', df.duplicated().sum())
# Избавляемся от явных дубликатов
df = df.drop_duplicates().reset_index(drop=True)

print(df['education'].value_counts())
df['education'] = df['education'].str.lower()
print()
# Смотрим как изменились данные после смены регистра.
print(df['education'].value_counts()) 
# Образовались новые явные дубликаты. Удаляем. Делаю повторно для наглядности.
print('Новых дубликатов:', df.duplicated().sum()) 
df = df.drop_duplicates().reset_index(drop=True)

Дубликатов явных: 54
среднее                13705
высшее                  4710
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      667
ВЫСШЕЕ                   273
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

среднее                15188
высшее                  5251
неоконченное высшее      743
начальное                282
ученая степень             6
Name: education, dtype: int64
Новых дубликатов: 17


<div class="alert alert-info">
<b>Комментарий:</b><br>
• После каждого шага вывожу количество явных дубликатов для отображения результатов замены неявных дубликатов.<br>
• Избавляюсь от неявных дубликатов после каждого появления, а не разом все.<br>
• Возможные причины неявных дубликатов: человеческий фактор при внесении данных.<br>
• Возможные причины явных дубликатов: повторное обращение клиента, некорректная выгрузка данных, повторное добавление записи сотрудником
</div>

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [7]:
# Создаем 2 новых датафрейма.
education_df = df[['education_id', 'education']]
family_df = df[['family_status_id', 'family_status']]
# Удаляем столбцы из исходного датафрейма.
df = df.drop(['education', 'family_status'], axis=1)
display(df.head(10))

,children,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,53,1,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,27,0,1,M,компаньон,0,255763,покупка жилья
6,0,43,0,0,F,компаньон,0,240525,операции с жильем
7,0,50,1,0,M,сотрудник,0,135823,образование
8,2,35,0,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи


### Шаг 2.6. Категоризация дохода.

In [8]:
# Функция для заполнения столбца категории доходов
def income_category(total):
    if total <= 30000:
        return 'E'
    if (total >= 30001) and (total <= 50000):
        return 'D'
    if (total >= 50001) and (total <= 200000):
        return 'C'
    if (total >= 200001) and (total <= 1000000):
        return 'B'
    if total >= 1000001:
        return 'A'

df['total_income_category'] = df['total_income'].apply(income_category)

### Шаг 2.7. Категоризация целей кредита.

In [9]:
realty = ('жиль', 'недвиж')
car = ('автомоб',)
education_str = ('образова',)
wedding = ('свадьб',)
categories = {
    realty: 'операции с недвижимостью',
    car: 'операции с автомобилем',
    education_str: 'получение образования',
    wedding: 'проведение свадьбы'
}
def category_for_purposes(purpose):
    for category_key in categories:
        for under_category_key in category_key:
            if under_category_key in purpose:
                try:
                    return categories[category_key]
                except:
                    return 'Такой категории нет'

                
df['purpose_category'] = df['purpose'].apply(category_for_purposes)
display(df.head(10))

,children,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
5,0,27,0,1,M,компаньон,0,255763,покупка жилья,B,операции с недвижимостью
6,0,43,0,0,F,компаньон,0,240525,операции с жильем,B,операции с недвижимостью
7,0,50,1,0,M,сотрудник,0,135823,образование,C,получение образования
8,2,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,C,проведение свадьбы
9,0,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью


### Ответы на вопросы.

##### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [10]:
# Выведем доли должников среди групп по количеству детей.
zero = 0
one = 0
two = 0
three = 0
four = 0
five = 0
child = 0

if child < 6:
    zero = (df[(df['children'] == child) & (df['debt'] == 0)]['children'].count()
        / df[df['children'] == child]['children'].count())
    child += 1
    one = (df[(df['children'] == child) & (df['debt'] == 0)]['children'].count()
        / df[df['children'] == child]['children'].count())
    child += 1
    two = (df[(df['children'] == child) & (df['debt'] == 0)]['children'].count()
        / df[df['children'] == child]['children'].count())
    child += 1
    three = (df[(df['children'] == child) & (df['debt'] == 0)]['children'].count()
        / df[df['children'] == child]['children'].count())
    child += 1
    four = (df[(df['children'] == child) & (df['debt'] == 0)]['children'].count()
        / df[df['children'] == child]['children'].count())
    child += 1
    five = (df[(df['children'] == child) & (df['debt'] == 0)]['children'].count()
        / df[df['children'] == child]['children'].count())

print('Доля клиентов, не имеющих задолженностей, относительно общего числа клиентов (группировка по количеству детей)')

debt_procent = ["{:.3%}".format(zero), "{:.3%}".format(one), "{:.3%}".format(two),
               "{:.3%}".format(three), "{:.3%}".format(four), "{:.3%}".format(five)]
pivot_children = df.pivot_table(index=['children'], values='debt')
pivot_children['refund'] = debt_procent
del pivot_children['debt']
display(pivot_children)

Доля клиентов, не имеющих задолженностей, относительно общего числа клиентов (группировка по количеству детей)


,refund
children,
0,92.474%
1,90.765%
2,90.508%
3,91.818%
4,90.244%
5,100.000%


##### Вывод 1: 

<div class="alert alert-info">
<b>Комментарий:</b><br>
Разобьем клиентов по количеству детей.<br>
Наиболее надежной группой являются клиенты с 5ю детьми. Доля возвратов: 100%<br>
Реже возвращают кредит клиенты с 4мя детьми. Доля возвратов: 90.244%
</div>

##### Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [11]:
# Выведем доли должников среди групп по семейному статусу.
married = 0
civil_marriage = 0
widow = 0
divorced = 0
not_merried = 0
family_status = 0

if family_status < 5:
    married = (df[(df['family_status_id'] == family_status) & (df['debt'] == 0)]['family_status_id'].count()
        / df[df['family_status_id'] == family_status]['family_status_id'].count())
    family_status += 1
    civil_marriage = (df[(df['family_status_id'] == family_status) & (df['debt'] == 0)]['family_status_id'].count()
        / df[df['family_status_id'] == family_status]['family_status_id'].count())
    family_status += 1
    widow = (df[(df['family_status_id'] == family_status) & (df['debt'] == 0)]['family_status_id'].count()
        / df[df['family_status_id'] == family_status]['family_status_id'].count())
    family_status += 1
    divorced = (df[(df['family_status_id'] == family_status) & (df['debt'] == 0)]['family_status_id'].count()
        / df[df['family_status_id'] == family_status]['family_status_id'].count())
    family_status += 1
    not_merried = (df[(df['family_status_id'] == family_status) & (df['debt'] == 0)]['family_status_id'].count()
        / df[df['family_status_id'] == family_status]['family_status_id'].count())

print('Доля клиентов, не имеющих задолженностей, относительно семейного статуса')
debt_procent = ["{:.3%}".format(married), "{:.3%}".format(civil_marriage), "{:.3%}".format(widow),
               "{:.3%}".format(divorced), "{:.3%}".format(not_merried)]
pivot_family_df = family_df.pivot_table(index=['family_status'])
pivot_family_df['refund'] = debt_procent
display(pivot_family_df)

Доля клиентов, не имеющих задолженностей, относительно семейного статуса


,family_status_id,refund
family_status,,
Не женат / не замужем,4,92.455%
в разводе,3,90.651%
вдовец / вдова,2,93.431%
гражданский брак,1,92.887%
женат / замужем,0,90.249%


##### Вывод 2: 

<div class="alert alert-info">
<b>Комментарий:</b><br>
Разобьем клиентов по семейному статусу.<br>
Наиболее надежной группой являются клиенты вдовцы/вдовы. Доля возвратов: 93.431%<br>
Реже возвращают кредит женатые/замужние. Доля возвратов: 90.249%
</div>

##### Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [12]:
# Выведем доли должников среди групп по уровню дохода.
cat_a = 0
cat_b = 0
cat_c = 0
cat_d = 0
cat_e = 0
category_number = df['total_income_category'].unique()

for categor in category_number:
    if categor == 'A':
        cat_a = (df[(df['total_income_category'] == categor) & (df['debt'] == 0)]['total_income_category'].count()
                 / df[df['total_income_category'] == categor]['total_income_category'].count())
    if categor == 'B':
        cat_b = (df[(df['total_income_category'] == categor) & (df['debt'] == 0)]['total_income_category'].count()
                 / df[df['total_income_category'] == categor]['total_income_category'].count())
    if categor == 'C':
        cat_c = (df[(df['total_income_category'] == categor) & (df['debt'] == 0)]['total_income_category'].count()
                 / df[df['total_income_category'] == categor]['total_income_category'].count())
    if categor == 'D':
        cat_d = (df[(df['total_income_category'] == categor) & (df['debt'] == 0)]['total_income_category'].count()
                 / df[df['total_income_category'] == categor]['total_income_category'].count())
    if categor == 'E':
        cat_e = (df[(df['total_income_category'] == categor) & (df['debt'] == 0)]['total_income_category'].count()
                 / df[df['total_income_category'] == categor]['total_income_category'].count())

print('Доля клиентов, не имеющих задолженностей, относительно статуса по уровню дохода')
debt_procent = ["{:.3%}".format(cat_a), "{:.3%}".format(cat_b), "{:.3%}".format(cat_c),
               "{:.3%}".format(cat_d), "{:.3%}".format(cat_e)]
pivot_total_income_category = df.pivot_table(index=['total_income_category'], values='debt')
pivot_total_income_category['refund'] = debt_procent
del pivot_total_income_category['debt']
display(pivot_total_income_category)

Доля клиентов, не имеющих задолженностей, относительно статуса по уровню дохода


,refund
total_income_category,
A,92.000%
B,92.937%
C,91.508%
D,94.000%
E,90.909%


##### Вывод 3: 

<div class="alert alert-info">
<b>Комментарий:</b><br>
Разобьем клиентов по уровню дохода.<br>
Наиболее надежными являются клиенты с уровнем дохода D.<br>
Наименее надежные - клиенты с категорией E.
</div>

##### Вопрос 4: Как разные цели кредита влияют на его возврат в срок?

In [14]:
print('Доля клиентов, не имеющих задолженностей, относительно цели для кредита')

pivot_purpose_table = pd.DataFrame()
#Количество вернувших кредит
pivot_purpose_table['sum'] = df.groupby('purpose_category')['debt'].count() - df.groupby('purpose_category')['debt'].sum()
pivot_purpose_table['count'] = df.groupby('purpose_category')['debt'].count()
pivot_purpose_table['refund'] = pivot_purpose_table['sum']/pivot_purpose_table['count']
display(pivot_purpose_table.sort_values('refund', ascending=False))

Доля клиентов, не имеющих задолженностей, относительно цели для кредита


,sum,count,refund
purpose_category,,,
операции с недвижимостью,10028,10810,0.927660
проведение свадьбы,2138,2324,0.919966
получение образования,3643,4013,0.907800
операции с автомобилем,3903,4306,0.906410


##### Вывод 4:

<div class="alert alert-info">
<b>Комментарий:</b><br>
Разобьем клиентов по цели для кредита.<br>
Наиболее надежные заемщики - клиенты, берущие кредит на операции с недвижимостью.<br>
Наименее надежные - на операции с автомобилем.
</div>

## Общий вывод:

<div class="alert alert-info">
При группировке клиентов по количеству детей можно сделать вывод, что наиболее надежными по возврату кредита клиентами являеются родители 5х детей. Доля возврата кредита по этой категории составляет 100%. Наименее надежная: родители 4х детей - 90.244%<br>
<br>
Наиболее надежной категорией среди клиентов по семейному статусу являются вдовы/вдовцы - 93.431%. Наименее надежные - женатые/замужние. Их доля составляет 90.249%.<br><br>
Наиболее надежной категорией среди клиентов по уровню дохода являются клиенты с уровенм дохода D - 94%. Наименее надежные - с категорией E. Их доля составляет 90.909%.<br><br>
Наиболее надежной категорией среди клиентов по целям для кредита являются те, кто берет кредит на операции с недвижимостью - 92.8%. Наименее надежные - категория с операциями с автомобилем. Их доля составляет 90.6%.
</div>